# LINGER ANALYSIS

## Preparation steps

In [2]:
%matplotlib inline
import os
os.chdir('/home/kl467102/thesis/')

In [26]:
import scanpy as sc
import scipy
import pandas as pd
import time
import os

In [27]:
os.chdir('/home/kl467102/thesis/linger_sandboxes/mock_10x_linger/')

In [28]:
input_dir = '../../data/filtered_feature_bc_matrix/mockified/' # path to 10x output files
label_file = '../../data/PBMC_label.txt' # path to cell-type annotations

The input consists of `matrix.mtx`, `features.tsv` and `barcodes.tsv` and an annotation file.

In [40]:
matrix=scipy.io.mmread(os.path.join(input_dir, 'matrix.mtx'))
features=pd.read_csv(os.path.join(input_dir, 'features.tsv'), sep='\t', header=None)
barcodes=pd.read_csv(os.path.join(input_dir, 'barcodes.tsv'), sep='\t', header=None)
label=pd.read_csv(label_file, sep='\t', header=0)

We have loaded the data, now we can inspect them.

In [41]:
matrix.shape

(73176, 11898)

In [42]:
features.shape

(73176, 6)

In [43]:
barcodes.shape

(11898, 1)

In [44]:
dense_matrix = matrix.toarray()
subset = dense_matrix[:5, :5]
subset

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [45]:
matrix

<73176x11898 sparse matrix of type '<class 'numpy.int64'>'
	with 31561199 stored elements in COOrdinate format>

In [35]:
pd.concat([features.head(2), features.tail(2)])

,0,1,2,3,4,5
0,ENSG00000243485,MIR1302-2HG,Gene Expression,chr1,29553,30267
1,ENSG00000237613,FAM138A,Gene Expression,chr1,36080,36081
73187,KI270734.1:130493-131463,KI270734.1:130493-131463,Peaks,KI270734.1,130493,131463
73188,KI270734.1:160749-161719,KI270734.1:160749-161719,Peaks,KI270734.1,160749,161719


In [36]:
barcodes.head(3)

,0
0,AAACAGCCAAGGAATC-1
1,AAACAGCCAATCCCTT-1
2,AAACAGCCAATGCGCT-1


In [37]:
label.head(3)

,barcode_use,label
0,AAACAGCCAAGGAATC-1,naive CD4 T cells
1,AAACAGCCAATCCCTT-1,memory CD4 T cells
2,AAACAGCCAATGCGCT-1,naive CD4 T cells


## INPUT MANIPULATION - we need to make it work

## OR DO WE? LATER

## Step 1 - preprocessing

### Extraction of matrices for RNASeq and ATACSeq

In [38]:
from LingerGRN.preprocess import *

In [39]:
adata_RNA, adata_ATAC = get_adata(matrix, features, barcodes, label)

ValueError: Length of values (73189) does not match length of index (73202)

In [19]:
adata_RNA

View of AnnData object with n_obs × n_vars = 9543 × 36601
    obs: 'barcode', 'sample', 'label', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'gene_ids', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [20]:
adata_ATAC

View of AnnData object with n_obs × n_vars = 9543 × 36601
    obs: 'barcode', 'sample', 'label'
    var: 'gene_ids'

### Removing low count cells

In [21]:
sc.pp.filter_cells(adata_RNA, min_genes=200)
sc.pp.filter_genes(adata_RNA, min_cells=3)
sc.pp.filter_cells(adata_ATAC, min_genes=200)
sc.pp.filter_genes(adata_ATAC, min_cells=3)
#peaks_to_remove = ['chr19:54136728-54137698', 'chr19:54155191-54156161', 'chr19:54905150-54906120']
#adata_ATAC = adata_ATAC[:, ~adata_ATAC.var['gene_ids'].isin(peaks_to_remove)].copy()



/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:139: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number
/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:139: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


In [22]:
adata_RNA

AnnData object with n_obs × n_vars = 9543 × 25485
    obs: 'barcode', 'sample', 'label', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes'
    var: 'gene_ids', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells'

In [23]:
adata_ATAC

AnnData object with n_obs × n_vars = 9539 × 15477
    obs: 'barcode', 'sample', 'label', 'n_genes'
    var: 'gene_ids', 'n_cells'

In [24]:
len(set(adata_ATAC.var['gene_ids'].values))

15462

In [25]:
adata_ATAC.var['gene_ids']

36604        chr1:119931-120901
36607        chr1:148706-149676
36615        chr1:777757-778727
36617        chr1:826597-827567
36625        chr1:958255-959225
                  ...          
73171                       NaN
73172                       NaN
73181    GL000219.1:82310-83280
73182    KI270711.1:23649-24619
73185      KI270721.1:1584-2554
Name: gene_ids, Length: 15477, dtype: object

In [33]:
adata_ATAC

AnnData object with n_obs × n_vars = 949 × 13850
    obs: 'barcode', 'sample', 'label', 'n_genes'
    var: 'gene_ids', 'n_cells'

In [34]:
# Find common barcodes between RNA and ATAC datasets
selected_barcode = list(set(adata_RNA.obs['barcode']) & set(adata_ATAC.obs['barcode']))

# Filter both datasets to keep only the shared barcodes
adata_RNA = adata_RNA[adata_RNA.obs['barcode'].isin(selected_barcode)]
adata_ATAC = adata_ATAC[adata_ATAC.obs['barcode'].isin(selected_barcode)]

In [35]:
adata_ATAC

View of AnnData object with n_obs × n_vars = 329 × 13850
    obs: 'barcode', 'sample', 'label', 'n_genes'
    var: 'gene_ids', 'n_cells'

In [36]:
adata_RNA

View of AnnData object with n_obs × n_vars = 329 × 1493
    obs: 'barcode', 'sample', 'label', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes'
    var: 'gene_ids', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells'

### Generating metacells

Metacells are generated to create more stable, representative profiles of groups of cells. 

In [37]:
from LingerGRN.pseudo_bulk import *

In [38]:
samplelist=list(set(adata_ATAC.obs['sample'].values)) 
tempsample=samplelist[0]
TG_pseudobulk=pd.DataFrame([])
RE_pseudobulk=pd.DataFrame([])

In [48]:
samplelist

[1]

In [41]:
adata_RNA.obs['sample'].unique().shape[0]*adata_RNA.obs['sample'].unique().shape[0]>100

False

In [39]:
singlepseudobulk = (adata_RNA.obs['sample'].unique().shape[0]*adata_RNA.obs['sample'].unique().shape[0]>100)
for tempsample in samplelist:
    adata_RNAtemp=adata_RNA[adata_RNA.obs['sample']==tempsample]
    adata_ATACtemp=adata_ATAC[adata_ATAC.obs['sample']==tempsample]
    TG_pseudobulk_temp,RE_pseudobulk_temp=pseudo_bulk(adata_RNAtemp,adata_ATACtemp,singlepseudobulk)                
    TG_pseudobulk=pd.concat([TG_pseudobulk, TG_pseudobulk_temp], axis=1)
    RE_pseudobulk=pd.concat([RE_pseudobulk, RE_pseudobulk_temp], axis=1)
    RE_pseudobulk[RE_pseudobulk > 100] = 100

/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:842: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:372: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:842: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [40]:
RE_pseudobulk

,GGTTATATCGTGCTTC-1,TCCCTCACATCATGGC-1,CCTCAAACATAGACCC-1,CTACTAAAGTTATGGC-1,TCTTGTCCATGATTGT-1,ACGCTTGAGGTCCTGA-1,CGATCCTTCAATGTCA-1,ACTAACTCATTGTCCT-1,CAAACTGGTGATTACG-1,CGACCTGCACTCAACA-1,...,GGCCAACTCGCTCCAT-1,CGCTTACTCCTAATTC-1,CCGTTACTCTGGCAAT-1,CACCTGTTCCTCCATA-1,GAACGAATCCTTAGTT-1,TATTACCTCTGGCATG-1,TCCTCAATCCTTTACG-1,GTTAAGCTCCTGTTCA-1,CGTAGTTAGCTTGCTC-1,ACTTGCACACGAATTT-1
chr8:20141307-20141916,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057822,0.000000,0.057822,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
chr20:24931527-24932323,0.115643,0.200351,0.057822,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.142529,0.000000,0.000000,0.000000,0.000000,0.057822,0.057822,0.000000
chr12:122681587-122682503,0.000000,0.000000,0.000000,0.000000,0.000000,0.200351,0.373816,0.258172,0.179010,0.258172,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
chr9:72393752-72394636,0.000000,0.000000,0.000000,0.000000,0.000000,0.289109,0.084707,0.094303,0.209947,0.289109,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
chr2:9238406-9239330,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.084707,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chr9:128370836-128371727,0.400701,0.587825,0.543230,0.485409,0.400701,0.477715,0.686179,0.468119,0.375730,0.415842,...,0.521890,0.400701,0.660788,0.558371,0.454472,0.548776,0.373816,0.543230,0.601052,0.315994
chr3:112332931-112333734,0.072963,0.200351,0.215492,0.200351,0.157670,0.000000,0.036481,0.000000,0.000000,0.000000,...,0.057822,0.200351,0.115643,0.200351,0.084707,0.084707,0.072963,0.200351,0.200351,0.157670
chr8:66092936-66093807,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057822,0.036481,0.057822,0.057822,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
chr14:91303014-91303930,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [46]:
13850 - 11758

2092

In [42]:
not_in_index = ['chr10:100198725-100199606', 'chr10:100445977-100446881', 'chr10:100656252-100656954', 'chr10:100933995-100934790', 'chr10:101394568-101395462', 'chr10:104027855-104028755', 'chr10:110107754-110108443', 'chr10:110464198-110465050', 'chr10:114108333-114109234', 'chr10:114427555-114428390', 'chr10:11451436-11452371', 'chr10:114693905-114694581', 'chr10:114703371-114704278', 'chr10:117006665-117007496', 'chr10:118704214-118705085', 'chr10:119054414-119055232', 'chr10:119237646-119238491', 'chr10:119369158-119369889', 'chr10:120824525-120825271', 'chr10:124262823-124263423', 'chr10:125792163-125793071', 'chr10:128030762-128031523', 'chr10:131964894-131965775', 'chr10:132665762-132666608', 'chr10:133530356-133531179', 'chr10:133608587-133609497', 'chr10:15245296-15246199', 'chr10:15261843-15262673', 'chr10:17842415-17843294', 'chr10:19834772-19835656', 'chr10:26647838-26648635', 'chr10:26701275-26702074', 'chr10:26829782-26830629', 'chr10:27546299-27546769', 'chr10:29703675-29704584', 'chr10:30399698-30400449', 'chr10:31862046-31862962', 'chr10:3221936-3222815', 'chr10:32433304-32434203', 'chr10:34814996-34815763', 'chr10:43600383-43601145', 'chr10:43624501-43625195', 'chr10:48661409-48662279', 'chr10:58177091-58177979', 'chr10:5903901-5904787', 'chr10:62517149-62518017', 'chr10:62533830-62534747', 'chr10:6516614-6517461', 'chr10:67520705-67521560', 'chr10:67735186-67736052', 'chr10:67902915-67903718', 'chr10:67931578-67932395', 'chr10:68870337-68871215', 'chr10:70903015-70903894', 'chr10:72208378-72209217', 'chr10:7233145-7234046', 'chr10:73425956-73426836', 'chr10:74750569-74751453', 'chr10:75011638-75012359', 'chr10:80226310-80227241', 'chr10:80234409-80235259', 'chr10:8033909-8034666', 'chr10:8114971-8115865', 'chr10:84347133-84347926', 'chr10:84355572-84356287', 'chr10:84424497-84425398', 'chr10:86899407-86900260', 'chr10:88491014-88491787', 'chr10:88743535-88744494', 'chr10:88979024-88979777', 'chr10:89003231-89003994', 'chr10:89322233-89323079', 'chr10:91000564-91001303', 'chr10:91245701-91246585', 'chr10:93751663-93752508', 'chr10:93826097-93826711', 'chr10:949364-950204', 'chr10:98167990-98168670', 'chr10:99766055-99766946', 'chr11:102539103-102539946', 'chr11:10408964-10409829', 'chr11:107591820-107592399', 'chr11:110208778-110209696', 'chr11:110248904-110249635', 'chr11:110280060-110280959', 'chr11:110410534-110411346', 'chr11:111397916-111398698', 'chr11:114603121-114603972', 'chr11:117865962-117866796', 'chr11:118021893-118022382', 'chr11:119594413-119595270', 'chr11:11960399-11961238', 'chr11:121387335-121388190', 'chr11:12143588-12144366', 'chr11:122616424-122617344', 'chr11:123073662-123074558', 'chr11:123234904-123235738', 'chr11:123707308-123708193', 'chr11:123710998-123711869', 'chr11:124481325-124482210', 'chr11:127073954-127074829', 'chr11:128602720-128603600', 'chr11:128956245-128957100', 'chr11:134196471-134197390', 'chr11:13450665-13451553', 'chr11:14695755-14696560', 'chr11:14861641-14862478', 'chr11:1639986-1640790', 'chr11:16812868-16813636', 'chr11:18578862-18579783', 'chr11:27497006-27497905', 'chr11:33301388-33302264', 'chr11:33332603-33333432', 'chr11:3404135-3404921', 'chr11:34058707-34059602', 'chr11:34936532-34937410', 'chr11:35282257-35283113', 'chr11:36343224-36344071', 'chr11:4076463-4077219', 'chr11:44523243-44524121', 'chr11:47466735-47467542', 'chr11:57276857-57277590', 'chr11:60192246-60192948', 'chr11:63212396-63213318', 'chr11:63950677-63951348', 'chr11:6526191-6527080', 'chr11:6673989-6674805', 'chr11:6812234-6813034', 'chr11:68690425-68691320', 'chr11:71383211-71384113', 'chr11:71423185-71424000', 'chr11:71458809-71459672', 'chr11:76614080-76614991', 'chr11:7892516-7893401', 'chr11:83007703-83008550', 'chr11:8343035-8343907', 'chr11:85780858-85781334', 'chr11:86818271-86819137', 'chr11:88324626-88325526', 'chr11:93236432-93237339', 'chr11:93728987-93729822', 'chr11:95239933-95240741', 'chr11:96357705-96358444', 'chr11:96463633-96464476', 'chr11:96464696-96465372', 'chr11:9729332-9730202', 'chr12:101318691-101319282', 'chr12:101709745-101710580', 'chr12:10279698-10280573', 'chr12:102957599-102958200', 'chr12:102964762-102965536', 'chr12:103518667-103519567', 'chr12:103698565-103699301', 'chr12:1046398-1047199', 'chr12:105383022-105383902', 'chr12:10582458-10583382', 'chr12:106112850-106113520', 'chr12:107032050-107032956', 'chr12:107360104-107360977', 'chr12:10770857-10771725', 'chr12:109115482-109116404', 'chr12:111556341-111557177', 'chr12:11598174-11598994', 'chr12:116136128-116136939', 'chr12:116141127-116141859', 'chr12:117501221-117502202', 'chr12:121756412-121757334', 'chr12:123482774-123483385', 'chr12:12430831-12431689', 'chr12:12631220-12631950', 'chr12:12684364-12685244', 'chr12:130918930-130919674', 'chr12:14463534-14464380', 'chr12:14756070-14756972', 'chr12:14809745-14810525', 'chr12:1554107-1555007', 'chr12:19451125-19452028', 'chr12:21677559-21678303', 'chr12:21940927-21941834', 'chr12:22109915-22110818', 'chr12:22329443-22330313', 'chr12:22596005-22596794', 'chr12:25212252-25213060', 'chr12:25341647-25342570', 'chr12:26931908-26932716', 'chr12:26963231-26964048', 'chr12:27260495-27261393', 'chr12:27824036-27824840', 'chr12:28253021-28253923', 'chr12:29216915-29217761', 'chr12:30853287-30854187', 'chr12:31353952-31354860', 'chr12:31661798-31662636', 'chr12:32020680-32021550', 'chr12:32051325-32052159', 'chr12:32671658-32672556', 'chr12:45773547-45774211', 'chr12:47094835-47095704', 'chr12:47143614-47144377', 'chr12:47686350-47687235', 'chr12:48445869-48446738', 'chr12:50784215-50785106', 'chr12:52672229-52673135', 'chr12:54270381-54271144', 'chr12:54949915-54950808', 'chr12:56670566-56671445', 'chr12:59678594-59679493', 'chr12:6051611-6052525', 'chr12:68200670-68201470', 'chr12:68386963-68387868', 'chr12:68579042-68579935', 'chr12:69906155-69907078', 'chr12:71694162-71695051', 'chr12:7537397-7538277', 'chr12:77049488-77050384', 'chr12:87848087-87848927', 'chr12:89440816-89441633', 'chr12:89736880-89737712', 'chr12:92010839-92011727', 'chr12:92084720-92085474', 'chr12:95653877-95654687', 'chr12:96419561-96420293', 'chr12:9685386-9686250', 'chr12:98492429-98493196', 'chr12:98744400-98745205', 'chr13:102457077-102457990', 'chr13:102660594-102661446', 'chr13:110941989-110942833', 'chr13:111342875-111343768', 'chr13:113005207-113006077', 'chr13:113875542-113876359', 'chr13:20136589-20137444', 'chr13:21240320-21241220', 'chr13:21625131-21626037', 'chr13:21676663-21677354', 'chr13:23321330-23322163', 'chr13:24020061-24020953', 'chr13:24554726-24555606', 'chr13:28497467-28498230', 'chr13:29801624-29802473', 'chr13:29835280-29836202', 'chr13:30253336-30254235', 'chr13:30629262-30630207', 'chr13:30648428-30649092', 'chr13:31773875-31774673', 'chr13:31827417-31828235', 'chr13:32407946-32408811', 'chr13:32837283-32838105', 'chr13:37056752-37057413', 'chr13:39615874-39616738', 'chr13:40570641-40571566', 'chr13:40734886-40735723', 'chr13:40792174-40793051', 'chr13:41361300-41362174', 'chr13:41486627-41487278', 'chr13:41919206-41920064', 'chr13:43085142-43086019', 'chr13:43963440-43964270', 'chr13:44908391-44909232', 'chr13:45473457-45474308', 'chr13:46726096-46726860', 'chr13:47036354-47037216', 'chr13:49421157-49422039', 'chr13:50846814-50847268', 'chr13:51001303-51002187', 'chr13:51955066-51955788', 'chr13:52846805-52847592', 'chr13:77492534-77493401', 'chr13:95221987-95222910', 'chr13:98991967-98992673', 'chr14:102871810-102872727', 'chr14:103185098-103185983', 'chr14:21719564-21720426', 'chr14:22468748-22469597', 'chr14:24464201-24465005', 'chr14:31325909-31326746', 'chr14:32043348-32044246', 'chr14:34498580-34499369', 'chr14:34562710-34563525', 'chr14:37197546-37198445', 'chr14:38256160-38256625', 'chr14:50973966-50974548', 'chr14:51202948-51203848', 'chr14:51563974-51564575', 'chr14:51566688-51567560', 'chr14:52683864-52684677', 'chr14:54532393-54533317', 'chr14:54757086-54757957', 'chr14:59175190-59176058', 'chr14:61578496-61579381', 'chr14:62078979-62079699', 'chr14:63711874-63712725', 'chr14:63748782-63749555', 'chr14:64194984-64195885', 'chr14:64429822-64430666', 'chr14:64733642-64734508', 'chr14:64935260-64936149', 'chr14:69755386-69756314', 'chr14:71529526-71530395', 'chr14:72776624-72777483', 'chr14:74928977-74929827', 'chr14:75372089-75372924', 'chr14:76726588-76727191', 'chr14:80977073-80977715', 'chr14:90761674-90762346', 'chr14:90910987-90911865', 'chr14:91029492-91030041', 'chr14:93159900-93160825', 'chr14:95707086-95707965', 'chr14:96198414-96199305', 'chr14:96553004-96553846', 'chr14:99388986-99389864', 'chr14:99746200-99747081', 'chr15:22774475-22775213', 'chr15:25838176-25839021', 'chr15:28912253-28913035', 'chr15:28986017-28986790', 'chr15:30621017-30621862', 'chr15:31294459-31295272', 'chr15:32657172-32658043', 'chr15:34284770-34285657', 'chr15:39404549-39405413', 'chr15:39478749-39479604', 'chr15:39744339-39745226', 'chr15:40258653-40259519', 'chr15:41465544-41466374', 'chr15:43473611-43474494', 'chr15:44447704-44448586', 'chr15:44788545-44789423', 'chr15:44801166-44802018', 'chr15:45338150-45339039', 'chr15:50774192-50775088', 'chr15:55829703-55830536', 'chr15:56208731-56209594', 'chr15:57166944-57167703', 'chr15:60697417-60698229', 'chr15:63096870-63097732', 'chr15:63440555-63441404', 'chr15:63524255-63524928', 'chr15:63751887-63752761', 'chr15:64464488-64465314', 'chr15:64784813-64785727', 'chr15:65684731-65685646', 'chr15:66401399-66402283', 'chr15:68631301-68632206', 'chr15:68741796-68742615', 'chr15:68839414-68840279', 'chr15:70853705-70854580', 'chr15:72342122-72343003', 'chr15:72630368-72631237', 'chr15:72721849-72722754', 'chr15:74302620-74303518', 'chr15:77353853-77354718', 'chr15:77973925-77974592', 'chr15:78024332-78025150', 'chr15:78951172-78952071', 'chr15:82266299-82267138', 'chr15:82852766-82853652', 'chr15:92655258-92656109', 'chr15:99123701-99124545', 'chr15:99810682-99811434', 'chr16:10275320-10276234', 'chr16:10312816-10313717', 'chr16:10381985-10382893', 'chr16:10497474-10498233', 'chr16:10504315-10505188', 'chr16:10987259-10988159', 'chr16:11958657-11959536', 'chr16:14487341-14488191', 'chr16:14911696-14912573', 'chr16:1637192-1638029', 'chr16:17491004-17491922', 'chr16:18776224-18777093', 'chr16:19601046-19601907', 'chr16:20798080-20798913', 'chr16:23156806-23157732', 'chr16:23914751-23915571', 'chr16:24955774-24956632', 'chr16:28569542-28570354', 'chr16:31327781-31328558', 'chr16:46801086-46801931', 'chr16:46910789-46911491', 'chr16:46916008-46916848', 'chr16:48479710-48480579', 'chr16:50081477-50082340', 'chr16:53445390-53446165', 'chr16:57701366-57702133', 'chr16:57731758-57732626', 'chr16:66457733-66458579', 'chr16:68698908-68699653', 'chr16:70254831-70255580', 'chr16:71777054-71777902', 'chr16:74403399-74404076', 'chr16:80569433-80570258', 'chr16:80719393-80720238', 'chr16:80727151-80728066', 'chr16:81576402-81577264', 'chr16:84227987-84228917', 'chr16:85715231-85716085', 'chr16:86509058-86509945', 'chr16:87550165-87551042', 'chr16:8908631-8909500', 'chr16:89944520-89945428', 'chr16:8996854-8997353', 'chr16:9068075-9068981', 'chr17:10052278-10053129', 'chr17:1216066-1216878', 'chr17:14118784-14119454', 'chr17:15341227-15342098', 'chr17:16872964-16873848', 'chr17:16911374-16912272', 'chr17:18418412-18419326', 'chr17:18926339-18927161', 'chr17:19442601-19443473', 'chr17:1977426-1978256', 'chr17:19912414-19913184', 'chr17:21422843-21423798', 'chr17:21544947-21545545', 'chr17:2195471-2196255', 'chr17:27360455-27361192', 'chr17:29909815-29910632', 'chr17:30663563-30664448', 'chr17:32002120-32002756', 'chr17:343048-343939', 'chr17:39376277-39377110', 'chr17:41354212-41355132', 'chr17:42640121-42640635', 'chr17:44288256-44289131', 'chr17:44878386-44879286', 'chr17:44941558-44942330', 'chr17:4573346-4574200', 'chr17:46082084-46082923', 'chr17:4649701-4650573', 'chr17:46748685-46749513', 'chr17:47275685-47276535', 'chr17:4771810-4772527', 'chr17:48573027-48573934', 'chr17:49434725-49435613', 'chr17:5329772-5330655', 'chr17:55463407-55464288', 'chr17:56934100-56934945', 'chr17:58121657-58122557', 'chr17:59812087-59812870', 'chr17:60517870-60518687', 'chr17:62552888-62553792', 'chr17:62720003-62720940', 'chr17:62884548-62885379', 'chr17:63188850-63189688', 'chr17:64794235-64795075', 'chr17:65774449-65775179', 'chr17:66799745-66800679', 'chr17:67200405-67201257', 'chr17:6853573-6854468', 'chr17:69444872-69445754', 'chr17:72930905-72931825', 'chr17:7673183-7673862', 'chr17:76786637-76787404', 'chr17:77185867-77186740', 'chr17:77343503-77344144', 'chr17:80019664-80020523', 'chr17:80112681-80113563', 'chr17:82150995-82151800', 'chr17:82737877-82738738', 'chr17:888687-889578', 'chr17:8904001-8904812', 'chr18:22221096-22221954', 'chr18:23428341-23429166', 'chr18:24094319-24095238', 'chr18:24475731-24476628', 'chr18:32139586-32140486', 'chr18:3384388-3385282', 'chr18:35525486-35526233', 'chr18:36149626-36150534', 'chr18:45139216-45140111', 'chr18:47951814-47952530', 'chr18:5196731-5197633', 'chr18:5305831-5306658', 'chr18:55274889-55275724', 'chr18:57784229-57785047', 'chr18:62339018-62339830', 'chr18:63283823-63284668', 'chr18:68735444-68736316', 'chr18:74242390-74243203', 'chr18:75478921-75479683', 'chr18:76426605-76427409', 'chr18:76990677-76991555', 'chr18:79062794-79063627', 'chr18:79525923-79526408', 'chr18:79755667-79756508', 'chr18:80154036-80154954', 'chr19:10067921-10068741', 'chr19:11606119-11606981', 'chr19:1199387-1200288', 'chr19:12899710-12900637', 'chr19:13011651-13012332', 'chr19:1301933-1302888', 'chr19:13505815-13506709', 'chr19:15212436-15213313', 'chr19:15780068-15780937', 'chr19:16502853-16503718', 'chr19:16970555-16971462', 'chr19:1701184-1702116', 'chr19:19686646-19687416', 'chr19:21697981-21698897', 'chr19:245349-246121', 'chr19:2576675-2577439', 'chr19:30001201-30002001', 'chr19:30005934-30006844', 'chr19:32586097-32586973', 'chr19:32813845-32814654', 'chr19:34261438-34262264', 'chr19:35048943-35049827', 'chr19:35060008-35060815', 'chr19:36483536-36484387', 'chr19:37303805-37304685', 'chr19:38351835-38352694', 'chr19:38513486-38514217', 'chr19:39067996-39068922', 'chr19:39710170-39711124', 'chr19:40907989-40908876', 'chr19:41172393-41173352', 'chr19:45706904-45707589', 'chr19:4894551-4895213', 'chr19:491807-492660', 'chr19:4941169-4942108', 'chr19:50142713-50143622', 'chr19:51448261-51449085', 'chr19:54089398-54090232', 'chr19:54425010-54425731', 'chr19:682723-683423', 'chr1:100151991-100152891', 'chr1:100219368-100220274', 'chr1:100475437-100476293', 'chr1:101070161-101071047', 'chr1:101331555-101332407', 'chr1:108214408-108215306', 'chr1:109649263-109650138', 'chr1:110258258-110259086', 'chr1:110746414-110747309', 'chr1:116232711-116233588', 'chr1:116789704-116790543', 'chr1:116971366-116972321', 'chr1:117017017-117017816', 'chr1:117093723-117094611', 'chr1:117389282-117389999', 'chr1:117558255-117559140', 'chr1:119008411-119009132', 'chr1:12029681-12030463', 'chr1:12265805-12266704', 'chr1:12572483-12573352', 'chr1:13790373-13791224', 'chr1:1411091-1411834', 'chr1:147586802-147587674', 'chr1:147717719-147718425', 'chr1:147931597-147932286', 'chr1:150758318-150759102', 'chr1:151419862-151420672', 'chr1:151606160-151607020', 'chr1:151716627-151717409', 'chr1:15277246-15278141', 'chr1:156310746-156311571', 'chr1:157708948-157709854', 'chr1:158353688-158354026', 'chr1:16048291-16049211', 'chr1:161426646-161427323', 'chr1:165367147-165368035', 'chr1:166083605-166084503', 'chr1:16700074-16700584', 'chr1:167654877-167655690', 'chr1:1696568-1697194', 'chr1:169670461-169671347', 'chr1:169881721-169882645', 'chr1:1699140-1700072', 'chr1:171557270-171558175', 'chr1:173930196-173931022', 'chr1:174413937-174414781', 'chr1:174872332-174873264', 'chr1:178899159-178900108', 'chr1:178925688-178926559', 'chr1:180975318-180976183', 'chr1:182340000-182340894', 'chr1:182633546-182634383', 'chr1:183048378-183049262', 'chr1:183596217-183597110', 'chr1:184788746-184789633', 'chr1:185647487-185648349', 'chr1:19113342-19114197', 'chr1:19128887-19129788', 'chr1:196608473-196609324', 'chr1:196660679-196661544', 'chr1:19858655-19859546', 'chr1:198609209-198610085', 'chr1:202376085-202376911', 'chr1:20255306-20256223', 'chr1:203428793-203429462', 'chr1:203807250-203808175', 'chr1:204327101-204327965', 'chr1:204466945-204467740', 'chr1:204791521-204792410', 'chr1:205199415-205200280', 'chr1:205823036-205823899', 'chr1:206540128-206540882', 'chr1:206674719-206675561', 'chr1:207711060-207711937', 'chr1:211399704-211400607', 'chr1:211478801-211479725', 'chr1:219194145-219194663', 'chr1:220085993-220086849', 'chr1:221066077-221066787', 'chr1:223707171-223708103', 'chr1:224261234-224262140', 'chr1:225343491-225344258', 'chr1:225915608-225916367', 'chr1:226149551-226150466', 'chr1:226954500-226955364', 'chr1:228635138-228636039', 'chr1:229410024-229410879', 'chr1:229542132-229543005', 'chr1:22979560-22980480', 'chr1:230084131-230084983', 'chr1:231644886-231645671', 'chr1:232690073-232690983', 'chr1:235140028-235140809', 'chr1:235461641-235462520', 'chr1:235986714-235987500', 'chr1:236865402-236866238', 'chr1:24003187-24004086', 'chr1:240539501-240540410', 'chr1:241985603-241986529', 'chr1:243723300-243724102', 'chr1:243938978-243939778', 'chr1:246788646-246789526', 'chr1:247646583-247647461', 'chr1:24834737-24835610', 'chr1:248686593-248687502', 'chr1:25148429-25149333', 'chr1:25601933-25602819', 'chr1:26370303-26371149', 'chr1:29282985-29283863', 'chr1:32112587-32113461', 'chr1:32431742-32432631', 'chr1:34781206-34782048', 'chr1:39032416-39033221', 'chr1:40364872-40365603', 'chr1:43041368-43042138', 'chr1:43545606-43546456', 'chr1:46424843-46425621', 'chr1:50333444-50333871', 'chr1:53498359-53499244', 'chr1:55039316-55040157', 'chr1:55375350-55376199', 'chr1:58929514-58930444', 'chr1:59947049-59947950', 'chr1:61120658-61121380', 'chr1:61160253-61161165', 'chr1:6180916-6181475', 'chr1:64246167-64246985', 'chr1:64538052-64538884', 'chr1:65793644-65794060', 'chr1:67112265-67113122', 'chr1:67359338-67360194', 'chr1:67733852-67734663', 'chr1:68253727-68254630', 'chr1:68259940-68260835', 'chr1:75726554-75727434', 'chr1:77347453-77348339', 'chr1:78836473-78837361', 'chr1:8047004-8047654', 'chr1:84387496-84388058', 'chr1:84565838-84566620', 'chr1:8473653-8474196', 'chr1:84930183-84931052', 'chr1:8499656-8500521', 'chr1:86741820-86742642', 'chr1:877203-878047', 'chr1:88935662-88936548', 'chr1:89644158-89645072', 'chr1:90015146-90016022', 'chr1:91487178-91488102', 'chr1:91512912-91513781', 'chr1:92323452-92324401', 'chr1:92367303-92368148', 'chr1:92929223-92930060', 'chr1:93642224-93643109', 'chr1:94008761-94009701', 'chr1:94978896-94979792', 'chr20:10551774-10552618', 'chr20:20463747-20464625', 'chr20:20547241-20548132', 'chr20:20767868-20768675', 'chr20:24012435-24013341', 'chr20:25514483-25515337', 'chr20:32314596-32315469', 'chr20:35129370-35130193', 'chr20:35238107-35238832', 'chr20:35250374-35251123', 'chr20:36676746-36677692', 'chr20:4187470-4188077', 'chr20:44288041-44288940', 'chr20:44447628-44448276', 'chr20:44810046-44810928', 'chr20:45629258-45630136', 'chr20:46246304-46247192', 'chr20:47258750-47259592', 'chr20:50446117-50446951', 'chr20:5150328-5151144', 'chr20:53034703-53035569', 'chr20:59093043-59093622', 'chr20:59235026-59235880', 'chr20:59385087-59386043', 'chr20:62511515-62512409', 'chr21:14115960-14116834', 'chr21:14253573-14254401', 'chr21:14518796-14519478', 'chr21:17615187-17615782', 'chr21:17829533-17830372', 'chr21:17900297-17901168', 'chr21:18443384-18444261', 'chr21:18460677-18461562', 'chr21:25423943-25424716', 'chr21:32731122-32731778', 'chr21:33955457-33956308', 'chr21:34078812-34079601', 'chr21:36630482-36631378', 'chr21:38817670-38818579', 'chr21:39143978-39144833', 'chr21:41801287-41802090', 'chr21:42989618-42990443', 'chr21:43697547-43698416', 'chr21:43991456-43992273', 'chr21:44697577-44698444', 'chr21:45269925-45270843', 'chr21:46065376-46066289', 'chr21:46388450-46389257', 'chr22:20741713-20742538', 'chr22:22032311-22033160', 'chr22:22058345-22059233', 'chr22:22318818-22319515', 'chr22:23034609-23035368', 'chr22:23902438-23903204', 'chr22:24264409-24265337', 'chr22:25033660-25034529', 'chr22:29419846-29420761', 'chr22:29635496-29636354', 'chr22:35694987-35695938', 'chr22:35921457-35922373', 'chr22:36799030-36799856', 'chr22:37045992-37046526', 'chr22:37391152-37392041', 'chr22:39877235-39878083', 'chr22:39885385-39886309', 'chr22:40593633-40594521', 'chr22:40714278-40715150', 'chr22:40798658-40799556', 'chr22:41758215-41758687', 'chr22:42139118-42139976', 'chr22:49723735-49724388', 'chr22:50774840-50775718', 'chr2:101240381-101241291', 'chr2:101486090-101486940', 'chr2:101768442-101769318', 'chr2:102753357-102754197', 'chr2:105691700-105692562', 'chr2:105782905-105783676', 'chr2:106205294-106206112', 'chr2:111930816-111931442', 'chr2:112242130-112242994', 'chr2:112528370-112529062', 'chr2:113096574-113097168', 'chr2:113704056-113704929', 'chr2:113791617-113792511', 'chr2:119415550-119416308', 'chr2:119921557-119922420', 'chr2:120441662-120442422', 'chr2:121472217-121473053', 'chr2:121902060-121902919', 'chr2:126689938-126690832', 'chr2:127478961-127479838', 'chr2:12760892-12761727', 'chr2:130228848-130229764', 'chr2:130873512-130874377', 'chr2:132267173-132267958', 'chr2:135525493-135526369', 'chr2:135773723-135774433', 'chr2:142914529-142915436', 'chr2:144377758-144378555', 'chr2:148781886-148782788', 'chr2:148789542-148790431', 'chr2:148956763-148957652', 'chr2:149561045-149561931', 'chr2:151280353-151281232', 'chr2:1550461-1551364', 'chr2:157363471-157364197', 'chr2:157777288-157778124', 'chr2:159149907-159150778', 'chr2:159157102-159158013', 'chr2:159619884-159620519', 'chr2:159637034-159637774', 'chr2:159863407-159864043', 'chr2:160194665-160195574', 'chr2:160647147-160647951', 'chr2:164976899-164977723', 'chr2:165594618-165595503', 'chr2:165825890-165826765', 'chr2:165950182-165951101', 'chr2:168001875-168002729', 'chr2:168136086-168136973', 'chr2:168545479-168546297', 'chr2:169562283-169563218', 'chr2:1709585-1710510', 'chr2:171141029-171141938', 'chr2:172674144-172674479', 'chr2:174346898-174347814', 'chr2:179021146-179021898', 'chr2:179194805-179195694', 'chr2:180991316-180992209', 'chr2:184766667-184767425', 'chr2:190471239-190472109', 'chr2:190662210-190663104', 'chr2:190870771-190871579', 'chr2:195583333-195584045', 'chr2:196935264-196936112', 'chr2:196959700-196960560', 'chr2:199822414-199823325', 'chr2:199849030-199849670', 'chr2:199932068-199932888', 'chr2:201652379-201653271', 'chr2:203551416-203552331', 'chr2:207225577-207226422', 'chr2:207337017-207337783', 'chr2:210198855-210199759', 'chr2:213101504-213102385', 'chr2:216849714-216850381', 'chr2:217864794-217865696', 'chr2:221548960-221549861', 'chr2:222426074-222426853', 'chr2:222849624-222850093', 'chr2:224427371-224428193', 'chr2:227456260-227456980', 'chr2:230454846-230455857', 'chr2:231236622-231237528', 'chr2:231260306-231261156', 'chr2:231852044-231852903', 'chr2:232333569-232334466', 'chr2:23351266-23352182', 'chr2:234061674-234062541', 'chr2:234166317-234167208', 'chr2:236359242-236360150', 'chr2:236503030-236503792', 'chr2:236607220-236608117', 'chr2:238434834-238435521', 'chr2:238448155-238449110', 'chr2:238784749-238785645', 'chr2:241366753-241367658', 'chr2:24575969-24576837', 'chr2:24813422-24814285', 'chr2:25787994-25788921', 'chr2:26397560-26398346', 'chr2:27948963-27949766', 'chr2:28573176-28574043', 'chr2:30541488-30542434', 'chr2:30765907-30766772', 'chr2:36333330-36334233', 'chr2:36663825-36664716', 'chr2:39494560-39495501', 'chr2:42513110-42513937', 'chr2:43315839-43316664', 'chr2:45734657-45735579', 'chr2:46719122-46720045', 'chr2:47735604-47736394', 'chr2:48398281-48399141', 'chr2:53928690-53929520', 'chr2:54081273-54082195', 'chr2:61580654-61581562', 'chr2:63902649-63903549', 'chr2:63993652-63994534', 'chr2:64403831-64404718', 'chr2:64939347-64940226', 'chr2:65388073-65388683', 'chr2:68031274-68032178', 'chr2:68487555-68488449', 'chr2:69456772-69457689', 'chr2:70864316-70865171', 'chr2:72144551-72145449', 'chr2:72150002-72150872', 'chr2:72925293-72926037', 'chr2:84314541-84315439', 'chr2:85487655-85488518', 'chr2:86138997-86139729', 'chr2:86851300-86852235', 'chr2:86854814-86855407', 'chr2:9215572-9216401', 'chr2:9512495-9513363', 'chr2:96482571-96483496', 'chr2:9650061-9650913', 'chr2:97005129-97005980', 'chr2:97951043-97951906', 'chr2:98593689-98594290', 'chr2:99085101-99085962', 'chr3:101414498-101415311', 'chr3:107530877-107531702', 'chr3:108052762-108053606', 'chr3:108669026-108669890', 'chr3:108777520-108778262', 'chr3:111529922-111530734', 'chr3:111571819-111572721', 'chr3:111593444-111594373', 'chr3:112494940-112495747', 'chr3:112751991-112752627', 'chr3:113063237-113064118', 'chr3:113217762-113218587', 'chr3:113760673-113761511', 'chr3:113802671-113803533', 'chr3:113910730-113911548', 'chr3:114207522-114208398', 'chr3:115652516-115653443', 'chr3:11799288-11800168', 'chr3:118938437-118939321', 'chr3:120100910-120101615', 'chr3:120145288-120146222', 'chr3:121810839-121811631', 'chr3:121841925-121842842', 'chr3:122307762-122308621', 'chr3:123570996-123571908', 'chr3:12400433-12401273', 'chr3:125383386-125384252', 'chr3:128273914-128274656', 'chr3:129468841-129469754', 'chr3:131490072-131491002', 'chr3:131629882-131630783', 'chr3:132443960-132444833', 'chr3:13262221-13263111', 'chr3:133591233-133592107', 'chr3:133880066-133880894', 'chr3:134344402-134345202', 'chr3:13932528-13933376', 'chr3:141274658-141275531', 'chr3:141386411-141387089', 'chr3:14215989-14216891', 'chr3:14278302-14278972', 'chr3:149267352-149268272', 'chr3:150774512-150775413', 'chr3:151186300-151187147', 'chr3:151196968-151197745', 'chr3:152443866-152444494', 'chr3:156043724-156044519', 'chr3:159861456-159862238', 'chr3:16414416-16415346', 'chr3:168141245-168142224', 'chr3:168213723-168214572', 'chr3:170375460-170375946', 'chr3:170768175-170769050', 'chr3:171001711-171002600', 'chr3:17637997-17638888', 'chr3:177106147-177107023', 'chr3:177320502-177321421', 'chr3:179090881-179091749', 'chr3:179353684-179354540', 'chr3:180960870-180961680', 'chr3:183870362-183871259', 'chr3:185207793-185208715', 'chr3:185613080-185613954', 'chr3:185645542-185646196', 'chr3:186528867-186529744', 'chr3:187679447-187680343', 'chr3:18988352-18989232', 'chr3:192816041-192816878', 'chr3:193589934-193590805', 'chr3:193742323-193743161', 'chr3:194270992-194271850', 'chr3:194548361-194549299', 'chr3:195170851-195171643', 'chr3:195690671-195691617', 'chr3:195833223-195834144', 'chr3:196885840-196886670', 'chr3:197256552-197257435', 'chr3:23256752-23257645', 'chr3:23758145-23759023', 'chr3:27668000-27668895', 'chr3:27685262-27686134', 'chr3:28050718-28051568', 'chr3:31003830-31004620', 'chr3:3119061-3119933', 'chr3:31472603-31473483', 'chr3:31903886-31904744', 'chr3:31947542-31948334', 'chr3:31985605-31986498', 'chr3:32091355-32092284', 'chr3:32097534-32098445', 'chr3:33377407-33378221', 'chr3:38376200-38376991', 'chr3:38999687-39000598', 'chr3:39085907-39086723', 'chr3:42048243-42049172', 'chr3:42263100-42263976', 'chr3:43759239-43760099', 'chr3:44058211-44059099', 'chr3:45529740-45530432', 'chr3:45701231-45701898', 'chr3:46027145-46028032', 'chr3:46294086-46294953', 'chr3:47142162-47142849', 'chr3:49068052-49068966', 'chr3:4928158-4928622', 'chr3:49606220-49606924', 'chr3:56605648-56606406', 'chr3:57174858-57175715', 'chr3:57647448-57648245', 'chr3:58899225-58900141', 'chr3:59001396-59002285', 'chr3:66821987-66822876', 'chr3:71082824-71083397', 'chr3:71854294-71855172', 'chr3:71872242-71872788', 'chr3:73056070-73056885', 'chr3:88000653-88001558', 'chr3:88218986-88219957', 'chr3:89107194-89108090', 'chr3:94071605-94072508', 'chr3:98536310-98537207', 'chr3:99823495-99824358', 'chr3:99866055-99866828', 'chr4:101355057-101355856', 'chr4:101852905-101853679', 'chr4:101862313-101863162', 'chr4:10192036-10192903', 'chr4:102262748-102263589', 'chr4:102447158-102448063', 'chr4:107035552-107036153', 'chr4:108519739-108520548', 'chr4:109658677-109659503', 'chr4:112261816-112262425', 'chr4:112765352-112766267', 'chr4:114070671-114071527', 'chr4:114601485-114602389', 'chr4:11467644-11468475', 'chr4:11505249-11506130', 'chr4:118419363-118420212', 'chr4:119296419-119297093', 'chr4:119366619-119367525', 'chr4:121036602-121037471', 'chr4:121074556-121075310', 'chr4:121126274-121127067', 'chr4:122534054-122534925', 'chr4:122576554-122577297', 'chr4:140353592-140354484', 'chr4:143417617-143418452', 'chr4:146241688-146242447', 'chr4:146275094-146275944', 'chr4:146333286-146334180', 'chr4:147794770-147795469', 'chr4:152763521-152764410', 'chr4:152957272-152958172', 'chr4:154329090-154329996', 'chr4:154416498-154417320', 'chr4:157220006-157220921', 'chr4:159206197-159207075', 'chr4:168515270-168516027', 'chr4:169720195-169721056', 'chr4:173181774-173182491', 'chr4:17385250-17386075', 'chr4:174419941-174420851', 'chr4:183447394-183448249', 'chr4:184526135-184526816', 'chr4:184838705-184839437', 'chr4:185134150-185134995', 'chr4:185258512-185259312', 'chr4:185352019-185352760', 'chr4:186137017-186137806', 'chr4:2103731-2104546', 'chr4:25783774-25784568', 'chr4:3010238-3011057', 'chr4:37773363-37774272', 'chr4:38151982-38152733', 'chr4:40066442-40067358', 'chr4:41413517-41414330', 'chr4:47533615-47534434', 'chr4:48296136-48296994', 'chr4:48403161-48404113', 'chr4:48681588-48682473', 'chr4:54144013-54144725', 'chr4:56479026-56479886', 'chr4:56668088-56668876', 'chr4:56749865-56750675', 'chr4:6191119-6191973', 'chr4:646212-647103', 'chr4:67572950-67573824', 'chr4:72567771-72568574', 'chr4:72569055-72569871', 'chr4:76430470-76431300', 'chr4:794977-795766', 'chr4:7967282-7968196', 'chr4:81003322-81004247', 'chr4:86894143-86894978', 'chr4:87919602-87920483', 'chr4:88028019-88028816', 'chr4:88734306-88735204', 'chr4:89069188-89070076', 'chr4:89353300-89354086', 'chr4:9000358-9001161', 'chr4:98288394-98289171', 'chr4:98544334-98545256', 'chr4:98612429-98613336', 'chr5:100841405-100842035', 'chr5:102548196-102549072', 'chr5:102867959-102868816', 'chr5:10288378-10289256', 'chr5:103245509-103246432', 'chr5:110756843-110757743', 'chr5:111249737-111250590', 'chr5:112934110-112934984', 'chr5:113273772-113274615', 'chr5:116137471-116138382', 'chr5:119059589-119060466', 'chr5:122735915-122736823', 'chr5:122770168-122771038', 'chr5:122902002-122902865', 'chr5:126961557-126962201', 'chr5:127757289-127758177', 'chr5:127892551-127893447', 'chr5:133105517-133106403', 'chr5:133139710-133140614', 'chr5:143383803-143384556', 'chr5:143421244-143422125', 'chr5:14483323-14484210', 'chr5:14528514-14529373', 'chr5:145946310-145947166', 'chr5:146466385-146467176', 'chr5:146795894-146796735', 'chr5:146965802-146966705', 'chr5:149617795-149618374', 'chr5:151427034-151427934', 'chr5:156891885-156892737', 'chr5:156956299-156957181', 'chr5:157560246-157561034', 'chr5:159002991-159003911', 'chr5:159179944-159180675', 'chr5:159826263-159827094', 'chr5:159965314-159966208', 'chr5:160488810-160489713', 'chr5:16555766-16556651', 'chr5:171409174-171410105', 'chr5:172424058-172424708', 'chr5:172868762-172869487', 'chr5:173572799-173573698', 'chr5:176041989-176042595', 'chr5:178068093-178068978', 'chr5:178097290-178098193', 'chr5:178547409-178548311', 'chr5:179343658-179344544', 'chr5:180304313-180305123', 'chr5:180373194-180374086', 'chr5:180432759-180433616', 'chr5:180607951-180608745', 'chr5:31854920-31855718', 'chr5:32514545-32515410', 'chr5:35771645-35772447', 'chr5:36177127-36177606', 'chr5:36558358-36559245', 'chr5:39086315-39087029', 'chr5:40480196-40481024', 'chr5:42748873-42749776', 'chr5:43060589-43061177', 'chr5:44875394-44876288', 'chr5:50512707-50513480', 'chr5:50607562-50608416', 'chr5:50699498-50700341', 'chr5:52946170-52946988', 'chr5:54234204-54235065', 'chr5:55905821-55906726', 'chr5:56350199-56350978', 'chr5:57095167-57095936', 'chr5:57323973-57324859', 'chr5:62443528-62444402', 'chr5:6370687-6371457', 'chr5:65814979-65815844', 'chr5:65984032-65984738', 'chr5:6758005-6758871', 'chr5:68264286-68265060', 'chr5:69023963-69024820', 'chr5:69125287-69126186', 'chr5:69250244-69251008', 'chr5:71446947-71447842', 'chr5:72194562-72195460', 'chr5:72268845-72269717', 'chr5:73454186-73455070', 'chr5:7377794-7378510', 'chr5:75091393-75092266', 'chr5:76431418-76432163', 'chr5:77627754-77628585', 'chr5:78339794-78340692', 'chr5:80686506-80687397', 'chr5:80993653-80994537', 'chr5:81533573-81534103', 'chr5:81908465-81909194', 'chr5:83092342-83093216', 'chr5:83427996-83428722', 'chr5:86665718-86666718', 'chr5:88979068-88979918', 'chr5:94692954-94693826', 'chr5:95358456-95359451', 'chr5:98942818-98943579', 'chr6:106609479-106610327', 'chr6:106614720-106615415', 'chr6:106676869-106677789', 'chr6:106702121-106703033', 'chr6:107638022-107638842', 'chr6:107642364-107643229', 'chr6:107781953-107782724', 'chr6:107891678-107892561', 'chr6:111692217-111693117', 'chr6:11444340-11445235', 'chr6:116218031-116218800', 'chr6:116697318-116698184', 'chr6:119267297-119268162', 'chr6:11931383-11932188', 'chr6:125753347-125754244', 'chr6:128478131-128479050', 'chr6:129661890-129662759', 'chr6:130114709-130115557', 'chr6:130116050-130116928', 'chr6:130947180-130948024', 'chr6:130980269-130981145', 'chr6:132551006-132551892', 'chr6:133889513-133890332', 'chr6:133969277-133970107', 'chr6:135537682-135538572', 'chr6:136920269-136921032', 'chr6:137105963-137106765', 'chr6:137939720-137940658', 'chr6:13830401-13831075', 'chr6:138762529-138763279', 'chr6:138818526-138819408', 'chr6:139109273-139110163', 'chr6:143596485-143597355', 'chr6:144418742-144419654', 'chr6:150417723-150418544', 'chr6:150765071-150765958', 'chr6:155185717-155186538', 'chr6:156728937-156729864', 'chr6:156755057-156755908', 'chr6:158451173-158451966', 'chr6:158705607-158706157', 'chr6:158707728-158708533', 'chr6:159132578-159133514', 'chr6:159904255-159905070', 'chr6:16640298-16641158', 'chr6:167031348-167032271', 'chr6:168370038-168370919', 'chr6:16964627-16965532', 'chr6:18044051-18044948', 'chr6:21597951-21598711', 'chr6:21667069-21667876', 'chr6:26109844-26110713', 'chr6:27553078-27553907', 'chr6:27592426-27593331', 'chr6:28346827-28347613', 'chr6:284915-285767', 'chr6:29564753-29565600', 'chr6:30019672-30020233', 'chr6:31235019-31235928', 'chr6:31238367-31239251', 'chr6:31252326-31253178', 'chr6:32201973-32202742', 'chr6:33093737-33094469', 'chr6:34635230-34636170', 'chr6:34917614-34918449', 'chr6:35703439-35704303', 'chr6:37285714-37286621', 'chr6:39209174-39209880', 'chr6:39263094-39264008', 'chr6:39530064-39530957', 'chr6:41177001-41177745', 'chr6:41503699-41504579', 'chr6:41625084-41625913', 'chr6:43807007-43807928', 'chr6:45930164-45931040', 'chr6:470758-471630', 'chr6:480321-481175', 'chr6:53552954-53553688', 'chr6:53741177-53742083', 'chr6:56247070-56247922', 'chr6:63808063-63808772', 'chr6:70598892-70599812', 'chr6:73231772-73232675', 'chr6:73557219-73558122', 'chr6:75201964-75202825', 'chr6:75593159-75593917', 'chr6:75823945-75824831', 'chr6:7978450-7979291', 'chr6:79862534-79863431', 'chr6:7991682-7992554', 'chr6:8017122-8018028', 'chr6:90108954-90109780', 'chr6:90320021-90320815', 'chr6:96532050-96532902', 'chr6:96907297-96908187', 'chr6:97284626-97285237', 'chr6:98827396-98828292', 'chr6:99329816-99330734', 'chr7:101507428-101508270', 'chr7:102577550-102578378', 'chr7:106116102-106116749', 'chr7:106732295-106732912', 'chr7:106788386-106789233', 'chr7:111089562-111090362', 'chr7:112764058-112764744', 'chr7:114889391-114890274', 'chr7:118153373-118154234', 'chr7:121360293-121361196', 'chr7:121372318-121373217', 'chr7:121373655-121374397', 'chr7:123701312-123702167', 'chr7:12388824-12389734', 'chr7:127477602-127478504', 'chr7:128071311-128072127', 'chr7:128793813-128794685', 'chr7:132653990-132654569', 'chr7:134522157-134522924', 'chr7:134593653-134594557', 'chr7:134946053-134946939', 'chr7:135186621-135187402', 'chr7:138170921-138171827', 'chr7:139582421-139583288', 'chr7:139583893-139584747', 'chr7:139708267-139709166', 'chr7:139837809-139838659', 'chr7:140102943-140103582', 'chr7:141071004-141071889', 'chr7:141590399-141591313', 'chr7:141668541-141669404', 'chr7:141876941-141877653', 'chr7:142185804-142186606', 'chr7:142620275-142621122', 'chr7:142700572-142701283', 'chr7:142723234-142723991', 'chr7:142732957-142733878', 'chr7:148842839-148843723', 'chr7:149439982-149440853', 'chr7:150551203-150551905', 'chr7:150891479-150892330', 'chr7:151095815-151096648', 'chr7:151700649-151701221', 'chr7:151958245-151959068', 'chr7:158609118-158610037', 'chr7:158613226-158614130', 'chr7:158662817-158663709', 'chr7:159207051-159208041', 'chr7:16346118-16346986', 'chr7:1717024-1717607', 'chr7:22665190-22666091', 'chr7:22765330-22766241', 'chr7:24283717-24284564', 'chr7:2457751-2458612', 'chr7:3026972-3027810', 'chr7:30410885-30411727', 'chr7:3082445-3083281', 'chr7:33012750-33013632', 'chr7:33738199-33739120', 'chr7:34718515-34719366', 'chr7:35714494-35715416', 'chr7:36143184-36143921', 'chr7:42105565-42106394', 'chr7:43763962-43764656', 'chr7:43778380-43779315', 'chr7:44196126-44197044', 'chr7:44454195-44454989', 'chr7:44534308-44535210', 'chr7:44786742-44787663', 'chr7:449170-450022', 'chr7:45920697-45921638', 'chr7:497877-498740', 'chr7:55432896-55433757', 'chr7:55553659-55554316', 'chr7:5739729-5740665', 'chr7:64947371-64947978', 'chr7:66324712-66325622', 'chr7:66718741-66719600', 'chr7:66770091-66770891', 'chr7:7236962-7237869', 'chr7:72954439-72955374', 'chr7:74138235-74139012', 'chr7:76122328-76123183', 'chr7:7772193-7773095', 'chr7:77747367-77748155', 'chr7:7805721-7806523', 'chr7:7829865-7830633', 'chr7:7856828-7857739', 'chr7:794687-795344', 'chr7:8047787-8048677', 'chr7:87204432-87205340', 'chr7:87586128-87586910', 'chr7:87589310-87590138', 'chr7:90665855-90666698', 'chr7:91912872-91913677', 'chr7:92877121-92877973', 'chr7:93039494-93040375', 'chr7:93176774-93177615', 'chr7:95715055-95715949', 'chr7:98221756-98222330', 'chr8:100974797-100975707', 'chr8:101091777-101092449', 'chr8:109592079-109592986', 'chr8:11507735-11508641', 'chr8:11566914-11567794', 'chr8:118024519-118024967', 'chr8:118958715-118959587', 'chr8:119206747-119207573', 'chr8:119934016-119934901', 'chr8:122678684-122679512', 'chr8:124363881-124364779', 'chr8:124896048-124896833', 'chr8:127670603-127671476', 'chr8:127740069-127740240', 'chr8:127919003-127919835', 'chr8:129696819-129697711', 'chr8:130286249-130287110', 'chr8:134700328-134700963', 'chr8:141079236-141080101', 'chr8:142658700-142659542', 'chr8:142684175-142685001', 'chr8:1800232-1801151', 'chr8:19456601-19457390', 'chr8:24260329-24261231', 'chr8:24299724-24300302', 'chr8:24309904-24310796', 'chr8:24326620-24327455', 'chr8:24912372-24912967', 'chr8:25228124-25228939', 'chr8:26243568-26244384', 'chr8:27499998-27500911', 'chr8:27748140-27749036', 'chr8:28075082-28075946', 'chr8:28336543-28337333', 'chr8:29164983-29165907', 'chr8:29237852-29238694', 'chr8:29405235-29406083', 'chr8:29974967-29975869', 'chr8:30046153-30046937', 'chr8:317318-318210', 'chr8:33497830-33498737', 'chr8:42524765-42525660', 'chr8:51873507-51874461', 'chr8:54010317-54011155', 'chr8:58413538-58414470', 'chr8:58724307-58725151', 'chr8:59045347-59046203', 'chr8:59068332-59069158', 'chr8:59096685-59097593', 'chr8:59147421-59148286', 'chr8:61124888-61125649', 'chr8:63007566-63008329', 'chr8:63179549-63180435', 'chr8:6558842-6559726', 'chr8:66477392-66478239', 'chr8:66555530-66556312', 'chr8:66589220-66590133', 'chr8:6723977-6724501', 'chr8:70633823-70634500', 'chr8:71874394-71875292', 'chr8:73684704-73685609', 'chr8:73917203-73918018', 'chr8:78806489-78807350', 'chr8:79612758-79613562', 'chr8:80078626-80079494', 'chr8:8723019-8723949', 'chr8:9007528-9008251', 'chr8:90082401-90083301', 'chr8:95174298-95175074', 'chr8:97736047-97736930', 'chr8:98040878-98041721', 'chr8:99702183-99703108', 'chr9:101573311-101574180', 'chr9:106921113-106922015', 'chr9:108821462-108822272', 'chr9:112028348-112029164', 'chr9:112924878-112925750', 'chr9:114156416-114157174', 'chr9:117546644-117547522', 'chr9:117772573-117773348', 'chr9:123200781-123201686', 'chr9:124818400-124819183', 'chr9:127283659-127284538', 'chr9:130358166-130359047', 'chr9:130447603-130448523', 'chr9:131343412-131344213', 'chr9:132213845-132214748', 'chr9:132804376-132805311', 'chr9:133040682-133041574', 'chr9:133646997-133647865', 'chr9:133894103-133894948', 'chr9:134301481-134302345', 'chr9:15228091-15228960', 'chr9:15285869-15286769', 'chr9:16867564-16868455', 'chr9:2047030-2047987', 'chr9:20740675-20741483', 'chr9:21149252-21150182', 'chr9:27483342-27484230', 'chr9:27655079-27655940', 'chr9:2765533-2766178', 'chr9:33276821-33277725', 'chr9:33322281-33323234', 'chr9:33617277-33618190', 'chr9:3379883-3380734', 'chr9:33883774-33884563', 'chr9:35629328-35630205', 'chr9:35741502-35742376', 'chr9:36985930-36986754', 'chr9:37925782-37926660', 'chr9:4615255-4616055', 'chr9:5263528-5264355', 'chr9:5265739-5266535', 'chr9:5827966-5828871', 'chr9:5853184-5854130', 'chr9:6695194-6696112', 'chr9:76415736-76416556', 'chr9:81637075-81637889', 'chr9:81727636-81728465', 'chr9:82184266-82185181', 'chr9:83934604-83935288', 'chr9:88543983-88544842', 'chr9:90724603-90725500', 'chr9:92103205-92103547', 'chr9:92541442-92542323', 'chr9:93276695-93277607', 'chr9:94346717-94347587', 'chr9:94802474-94803321', 'chr9:95896277-95897136', 'chr9:96203618-96204500', 'chr9:97703515-97704407', 'chr9:99154343-99155062', 'chrX:10003453-10004366', 'chrX:101056810-101057700', 'chrX:101693270-101694139', 'chrX:103446684-103447561', 'chrX:103711256-103712138', 'chrX:109794017-109794920', 'chrX:119579192-119580079', 'chrX:119655064-119655859', 'chrX:119711659-119712541', 'chrX:119770833-119771704', 'chrX:124266501-124267384', 'chrX:12997733-12998542', 'chrX:133334802-133335675', 'chrX:134338222-134339093', 'chrX:134483743-134484645', 'chrX:135931600-135932504', 'chrX:136200199-136201098', 'chrX:148562625-148563527', 'chrX:149831594-149832253', 'chrX:150543055-150543822', 'chrX:153056052-153056903', 'chrX:153226232-153227133', 'chrX:153888111-153889026', 'chrX:1552802-1553576', 'chrX:15564923-15565835', 'chrX:155842904-155843730', 'chrX:155874362-155875206', 'chrX:17374603-17375414', 'chrX:17553891-17554815', 'chrX:19593420-19594290', 'chrX:20116239-20117088', 'chrX:20248377-20249257', 'chrX:24241986-24242515', 'chrX:24497872-24498468', 'chrX:30562805-30563630', 'chrX:378273-379142', 'chrX:38784481-38785387', 'chrX:41045885-41046634', 'chrX:41127265-41128141', 'chrX:41906455-41907360', 'chrX:42093658-42094341', 'chrX:44040098-44040996', 'chrX:46443687-46444545', 'chrX:46485612-46486501', 'chrX:47156565-47157426', 'chrX:48660541-48661471', 'chrX:51332291-51333213', 'chrX:53748007-53748930', 'chrX:55688280-55689118', 'chrX:55698315-55699141', 'chrX:57587239-57588087', 'chrX:64850402-64851303', 'chrX:65752180-65753069', 'chrX:65924645-65925395', 'chrX:7090476-7091351', 'chrX:7091938-7092526', 'chrX:70998222-70999090', 'chrX:71599739-71600443', 'chrX:7372691-7373498', 'chrX:76420026-76420847', 'chrX:79194735-79195595', 'chrX:79403014-79403922', 'chrX:8815152-8815983', 'chrX:9534889-9535771']

In [45]:
len(not_in_index)

1596

Write preprocessed data to a file:

In [206]:
linger_storage = 'hhep_notebook/results/hhep/linger_storage'
os.makedirs(linger_storage, exist_ok = True)
os.makedirs('hhep_notebook/data', exist_ok = True)

adata_ATAC.write(os.path.join(linger_storage,'adata_ATAC.h5ad'))
adata_RNA.write(os.path.join(linger_storage,'adata_RNA.h5ad'))
TG_pseudobulk=TG_pseudobulk.fillna(0)
RE_pseudobulk=RE_pseudobulk.fillna(0)
pd.DataFrame(adata_ATAC.var['gene_ids']).to_csv('hhep_notebook/data/Peaks.txt',header=None,index=None) #hard coded path to Peaks in preprocess
TG_pseudobulk.to_csv(os.path.join(linger_storage,'TG_pseudobulk.tsv'))
RE_pseudobulk.to_csv(os.path.join(linger_storage,'RE_pseudobulk.tsv'))

## Step 2 - training

In [207]:
method = 'LINGER'
Datadir = '/home/kl467102/proj_v0/downloads/' # Directory for the downloaded general gene regulatory network for LINGER
GRNdir = Datadir+'data_bulk/'
genome = 'hg38'
outdir = os.path.join(os.getcwd(),'hhep_notebook/results/')
os.makedirs(outdir, exist_ok = True)

NOTE TO SELF: Because of `pybedtools` dependency `outdir` needs to be specified as an absolute path, otherwise resulting in an error

In [208]:
import importlib
import LingerGRN
from LingerGRN import preprocess
importlib.reload(preprocess)

<module 'LingerGRN.preprocess' from '/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/LingerGRN/preprocess.py'>

In [209]:
from LingerGRN.preprocess import *

In [210]:
preprocess(TG_pseudobulk,RE_pseudobulk,GRNdir,genome,method,outdir)

Mapping gene expression...
Generate TF expression...
Generate RE chromatin accessibility...
Generate TF binding...


100%|███████████████████████████████████████████████████████████████████████████████████| 23/23 [17:20<00:00, 45.25s/it]


KeyError: "None of [Index(['chr10:100148358-100149290', 'chr10:100167414-100168270',\n       'chr10:100240521-100241436', 'chr10:100373983-100374252',\n       'chr10:101012607-101013519', 'chr10:101032058-101032874',\n       'chr10:101226641-101227454', 'chr10:101236211-101237083',\n       'chr10:101310349-101311156', 'chr10:101569949-101570838',\n       ...\n       'chrX:84409649-84410534', 'chrX:8782902-8783752',\n       'chrX:8980577-8981392', 'chrX:8987452-8988337', 'chrX:9456304-9457207',\n       'chrX:9465866-9466676', 'chrX:9515757-9516646', 'chrX:9534889-9535771',\n       'chrX:9565825-9566739', 'chrX:9907273-9908109'],\n      dtype='object', length=12631)] are in the [index]"

In [142]:
import LingerGRN.LINGER_tr as LINGER_tr

In [179]:
import importlib
import LingerGRN
from LingerGRN import LINGER_tr
importlib.reload(LINGER_tr)


<module 'LingerGRN.LINGER_tr' from '/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/LingerGRN/LINGER_tr.py'>

In [ ]:
activef='ReLU'
LINGER_tr.training(GRNdir,method,outdir,activef,'Human')

        

chr1


  0%|                                                                                          | 0/1519 [00:00<?, ?it/s]

['459_1460_4096_4096_6537_6537_6537_6625_6625_9489' '363_3361' '363_3361'
 ... '9949' '264_1671_4733_9775' nan]
[[0.46769622 0.4454923  0.6338216  ... 0.6678138  0.64416456 0.5666959 ]
 [0.18106914 0.21080479 0.25053784 ... 0.16499498 0.28448483 0.191066  ]
 [0.16861826 0.10074462 0.23438916 ... 0.22498614 0.25003755 0.3072867 ]
 ...
 [0.5099761  0.2732     0.7682441  ... 0.6377838  0.76048076 0.70852846]
 [0.43522528 0.42256272 0.48136303 ... 0.6836907  0.62508106 0.53608805]
 [0.3576924  0.22600096 0.57979745 ... 0.53142583 0.55257004 0.46081558]]
555
[[0.38615456 0.0578217  0.23063298 0.29185143 0.25197324 0.39235368
  0.49220216 0.14592572 0.10944429 0.46531656 0.23063298 0.334532
  0.4350342  0.13078456 0.3827578  0.334532   0.3710134  0.57476085
  0.22508769 0.21888858 0.27671027 0.28845468 0.14592572 0.20374742
  0.36141753 0.42263603 0.3710134  0.10944429]
 [0.27671027 0.42263603 0.334532   0.334532   0.48665679 0.23063298
  0.42883512 0.5101456  0.24022885 0.35587224 0.3676166

  0%|                                                                                | 1/1519 [00:02<1:02:27,  2.47s/it]

X_tr.shape=torch.Size([28, 557])
['459_1460_4096_4096_6537_6537_6537_6625_6625_9489' '363_3361' '363_3361'
 ... '9949' '264_1671_4733_9775' nan]
[[0.46769622 0.4454923  0.6338216  ... 0.6678138  0.64416456 0.5666959 ]
 [0.18106914 0.21080479 0.25053784 ... 0.16499498 0.28448483 0.191066  ]
 [0.16861826 0.10074462 0.23438916 ... 0.22498614 0.25003755 0.3072867 ]
 ...
 [0.5099761  0.2732     0.7682441  ... 0.6377838  0.76048076 0.70852846]
 [0.43522528 0.42256272 0.48136303 ... 0.6836907  0.62508106 0.53608805]
 [0.3576924  0.22600096 0.57979745 ... 0.53142583 0.55257004 0.46081558]]
555
[[0.42883512 0.3131917  0.31939083 0.3710134  0.167266   0.30979496
  0.34967315 0.14592572 0.15767013 0.42263603 0.21888858 0.334532
  0.29805055 0.21549182 0.63878167 0.18240716 0.334532   0.20374742
  0.24022885 0.44612482 0.22508769 0.48665679 0.42263603 0.20374742
  0.27671027 0.27331352 0.2615691  0.334532  ]]
1
inputs.shape=torch.Size([556, 28])


...

X_tr.shape=torch.Size([28, 556])
['459_1460_4096_4096_6537_6537_6537_6625_6625_9489' '363_3361' '363_3361'
 ... '9949' '264_1671_4733_9775' nan]
[[0.46769622 0.4454923  0.6338216  ... 0.6678138  0.64416456 0.5666959 ]
 [0.18106914 0.21080479 0.25053784 ... 0.16499498 0.28448483 0.191066  ]
 [0.16861826 0.10074462 0.23438916 ... 0.22498614 0.25003755 0.3072867 ]
 ...
 [0.5099761  0.2732     0.7682441  ... 0.6377838  0.76048076 0.70852846]
 [0.43522528 0.42256272 0.48136303 ... 0.6836907  0.62508106 0.53608805]
 [0.3576924  0.22600096 0.57979745 ... 0.53142583 0.55257004 0.46081558]]
555
ok1
inputs.shape=torch.Size([555, 28])


 50%|███████████████████████████████████████▉                                        | 759/1519 [36:15<30:10,  2.38s/it]

X_tr.shape=torch.Size([28, 555])
['459_1460_4096_4096_6537_6537_6537_6625_6625_9489' '363_3361' '363_3361'
 ... '9949' '264_1671_4733_9775' nan]
[[0.46769622 0.4454923  0.6338216  ... 0.6678138  0.64416456 0.5666959 ]
 [0.18106914 0.21080479 0.25053784 ... 0.16499498 0.28448483 0.191066  ]
 [0.16861826 0.10074462 0.23438916 ... 0.22498614 0.25003755 0.3072867 ]
 ...
 [0.5099761  0.2732     0.7682441  ... 0.6377838  0.76048076 0.70852846]
 [0.43522528 0.42256272 0.48136303 ... 0.6836907  0.62508106 0.53608805]
 [0.3576924  0.22600096 0.57979745 ... 0.53142583 0.55257004 0.46081558]]
555
ok1
inputs.shape=torch.Size([555, 28])


 50%|████████████████████████████████████████                                        | 760/1519 [36:17<29:37,  2.34s/it]

X_tr.shape=torch.Size([28, 555])
['459_1460_4096_4096_6537_6537_6537_6625_6625_9489' '363_3361' '363_3361'
 ... '9949' '264_1671_4733_9775' nan]
[[0.46769622 0.4454923  0.6338216  ... 0.6678138  0.64416456 0.5666959 ]
 [0.18106914 0.21080479 0.25053784 ... 0.16499498 0.28448483 0.191066  ]
 [0.16861826 0.10074462 0.23438916 ... 0.22498614 0.25003755 0.3072867 ]
 ...
 [0.5099761  0.2732     0.7682441  ... 0.6377838  0.76048076 0.70852846]
 [0.43522528 0.42256272 0.48136303 ... 0.6836907  0.62508106 0.53608805]
 [0.3576924  0.22600096 0.57979745 ... 0.53142583 0.55257004 0.46081558]]
555
[[0.39235368 0.21888858 0.29805055 0.22508769 0.32833287 0.5961011
  0.167266   0.32833287 0.29805055 0.24022885 0.22508769 0.3791266
  0.44397625 0.3131917  0.03648143 0.3131917  0.10944429 0.334532
  0.37912664 0.18240716 0.36141753 0.167266   0.334532   0.2829094
  0.3131917  0.3710134  0.30424967 0.20374742]
 [0.39235368 0.21888858 0.29805055 0.22508769 0.32833287 0.5961011
  0.167266   0.32833287 0.

## Step 3 - population GNR inference

In [ ]:
import LingerGRN.LL_net as LL_net
LL_net.TF_RE_binding(GRNdir,adata_RNA,adata_ATAC,genome,method,outdir)

In [ ]:
LL_net.cis_reg(GRNdir,adata_RNA,adata_ATAC,genome,method,outdir)

In [ ]:
LL_net.trans_reg(GRNdir,method,outdir,genome)

## Step 4 - cell-type specific GNR

In [ ]:
celltype='all'

In [ ]:
LL_net.cell_type_specific_TF_RE_binding(GRNdir,adata_RNA,adata_ATAC,genome,celltype,outdir,method)

In [ ]:
LL_net.cell_type_specific_cis_reg(GRNdir,adata_RNA,adata_ATAC,genome,celltype,outdir)

In [ ]:
LL_net.cell_type_specific_trans_reg(GRNdir,adata_RNA,celltype,outdir)

# Session information

In [27]:
import session_info
session_info.show()